### Imports

In [56]:
import requests
import json
import ee
import geemap
import folium
import os
import pandas as pd
import glob
import time
import math
import numpy as np
from datetime import datetime

import modules.area_stats as area_stats
import modules.WDPA_prep as WDPA_prep
import modules.image_prep as image_prep
import modules.agstack_to_gee as agstack_to_gee
import modules.tidy_tables as tidy_tables
# from modules.agstack_to_gee import geo_id_list_to_feature_collection ,geo_id_to_feature,json_to_feature_with_id,geo_id_to_json
# from modules.agstack_to_gee import *##vgeo_id_list_to_feature_collection ,geo_id_to_feature,json_to_feature_with_id,geo_id_to_json

print("imports complete")

imports complete


In [57]:
ee.Initialize()

### Parameters

#### Web app deployed URLs

In [58]:
asset_registry_base = "https://api-ar.agstack.org"
user_registry_base = "https://user-registry.agstack.org"

#### Add your email & password to register with Agstack and test out the APIs

In [59]:
email = "remi.dannunzio@fao.org"
phone_num = "0123456789"
password = "Remi12345"
discoverable = True

##appears to not need proper credentials for running this...

In [60]:
out_path = os.path.join('/home/sepal-user/fdap/')

#long csv
out_long_csv_name = 'stats_long_format_continuous_data.csv' 

dataset_column_name = "dataset_name"

#wide csv
out_file_long=out_path+out_long_csv_name

out_wide_csv_name = 'stats_wide_format_continuous_data.csv' #set output name

out_file_wide = out_path+out_wide_csv_name #set full path for output csv


In [61]:
#output column names
geometry_area_column = "Geometry_area_km2"

geo_id_column = "Geo_id"

#debug printing
verbose = True

## stats by pixel_area
to_pixel_area = True

## reducer choice
reducer_choice = ee.Reducer.sum().combine(
  reducer2=ee.Reducer.count(), sharedInputs=True)

#ee.Reducer.sum()


Local deforestation alert parameters

In [62]:
# Define how many days back
how_many_days_back = -(365*2)  # must be negative

#define buffer distance (m)
local_alerts_buffer_radius = 2000 


For defining random polygons (TEMPORARY ONLY - can tidy if needed for longer termn testing)

In [63]:
#if true will make number_of_points per admin boundary (i.e. no of points x no of boundaries), as opposed tob spread across them
points_per_admin_boundary = False 

#how many random admin boundaries do we select
number_of_boundaries = 40

#how many points  
number_of_points = 40

#buffer
buffer_distance_meters = 500

#max error in m (coarse is quicker to run) 
max_error_alert_buff = 100 

seed = 18 # so it can be reproducable - each number is a new random combination                                                                                                
   
max_error = 1000 #in meters for admin boundaries (can be coarse ot speed up computation, as just example)


### 2. Fetch some fields (public)

#### Transform geometries into a feature collection
NB Currently not working due to session issues

In [64]:
CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
'97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

all_geo_ids= CIV_ids+GHA_ids+IDN_ids

print (all_geo_ids)

['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918', 'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f', 'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3', 'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f', '88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c', '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb', 'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7', '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45', '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']


#### Start Session

NB this is timing out so skipping section - seems to work without somehow. Maybe an open connection already...

In [10]:
## using session to store cookies that are persistent
session = requests.session()
session.headers = headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
req_body = {'email': email, 'password': password}
res = session.post(user_registry_base, json=req_body)
print(session.cookies)
print(res.status_code)

<RequestsCookieJar[]>
500


In [11]:
roi = agstack_to_gee.geo_id_list_to_feature_collection(all_geo_ids,geo_id_column, session,asset_registry_base)

#checks 
print ("Count of geo ids in list: ", len(all_geo_ids))
print ("Count of features in FeatureCollection: ", roi.size().getInfo())

Count of geo ids in list:  11
Count of features in FeatureCollection:  11


##### Alternative feature collection: create random polygons
select random points inside administrative boundaries and buffer them 

In [12]:
# def create_random_points_in_polys(feature): #to tidy
#     """ creates random points within either a polygon or a feature collection NB relies upon some globals being set currently"""
#     return ee.FeatureCollection.randomPoints(region = feature.geometry(max_error), points = number_of_points, seed=seed, maxError=10)

# admin_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2").filter(
#     ee.Filter.inList("ADM0_NAME",["Indonesia","Malaysia","Ghana"]))##.filter(ee.Filter.lt("Shape_Area",1000))

# random_collection = admin_boundaries.randomColumn(seed = seed).sort('random').limit(number_of_boundaries)

# if points_per_admin_boundary:
#     random_points = random_collection.map(create_random_points_in_polys).flatten()
# else:
#     random_points = create_random_points_in_polys(random_collection)

# random_buffers = random_points.map(lambda feature: feature.buffer(buffer_distance_meters,100)) ##buffer by distance in meters

# roi= random_buffers.map(lambda feature: feature.set(geo_id_column,(feature.get("system:index"))))
# # roi= random_buffers.map(set_geo_id_from_system_index) ## add surrogate "geo_id" for formatting

# #checks
# if verbose:
#     print("number of admin regions",random_collection.size().getInfo())
#     print("number of countries",random_collection.aggregate_array("ADM0_NAME").distinct().length().getInfo())
#     print("number of countries",random_collection.aggregate_array("ADM0_NAME").distinct().getInfo())
#     print("number of buffers created",random_buffers.size().getInfo())
#     # geemap.ee_to_pandas(roi)###create random points and buffer them


In [13]:
# ## checks
Map=geemap.Map()
Map.addLayer(roi, "", 'roi', 1, 1)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### Feature prep
- Create additional buffer zones for deforestation risk 
- Add area property to feature(s) 
- Select only columns of interest

In [14]:
roi = area_stats.add_area_km2_property_to_feature_collection(roi,geometry_area_column)

roi  = roi.select([geometry_area_column,geo_id_column]) ##select only fields of interest

roi_alerts_buffer = roi.map(lambda feature: 
        feature.buffer(local_alerts_buffer_radius,max_error_alert_buff))

## to check HOW TO HANDLE ALERT BUFFER - best ignore until know how/if alerts will be used like this
###roi_alerts_buffer = roi_alerts_buffer.map(add_area_km2_property_to_feature).select([geometry_area_column,geo_id_column])

geemap.ee_to_pandas(roi_alerts_buffer)
geemap.ee_to_pandas(roi)

,Geo_id,Geometry_area_km2
0,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,0.083162
1,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...,0.019902
2,b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0...,0.038155
3,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,0.036282
4,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,0.019471
5,ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e45485042...,0.041720
6,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,0.166761
7,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,0.313534
8,1a41a309ae2387f36a604c9a6c81887e64357a7f61d228...,0.019736
9,1a4472dc40700ef33f931863f58d444f243d6441861667...,0.127822


In [15]:
# ## checks
# Map=geemap.Map()
# Map.addLayer(roi, "", 'roi', 1, 1)
# Map.addLayer(roi_alerts_buffer, "", 'roi_alerts_buffer', 1, 1)
# Map

### 1. Grab land cover datasets

#### Fetch: GLAD: Global 2000-2020 Land Cover and Land Use Change

For full legend see: https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx

NB all tree cover have values related to height: from 3m to >25m. 
stable tree height values: 25-48 (terra firma); 125-148 (wetland);
height after disturbance values: 49-72 (terra firma); 149-172 (wetland) 
tree gain height values: 73-96 (terra firma); 173-196 (wetland); 

for now including all heights until agreed forest definition 

- value 25 tree stable
- value 49 tree after disturbance 
- value 73 tree gain
- value 248 tree from crop (NB need to investigate: likely plantation but could be secondary)
- value 245 crop
- value 24 vegetation 
- value 200 water 
- value 250 built up


In [16]:
glad_landcover_2020 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020')
landmask = ee.Image("projects/glad/OceanMask").lte(1)
glad_landcover_2020 = glad_landcover_2020.updateMask(landmask);

#trees
glad_landcover_2020_main = glad_landcover_2020.where((glad_landcover_2020.gte(25)).And(glad_landcover_2020.lte(48)), 25) #stable trees
glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(125)).And(glad_landcover_2020.lte(148)), 25) #stable trees

# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(49)).And(glad_landcover_2020.lte(72)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(149)).And(glad_landcover_2020.lte(172)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(73)).And(glad_landcover_2020.lte(96)), 73)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(173)).And(glad_landcover_2020.lte(196)), 73)

# #vegetation
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(0)).And(glad_landcover_2020.lte(24)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(100)).And(glad_landcover_2020.lte(124)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.eq(240)).Or(glad_landcover_2020.eq(249)), 24)

# #cropland
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(244)).And(glad_landcover_2020.lte(247)), 244)

# #water
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(200)).And(glad_landcover_2020.lte(211)), 200)
# glad_landcover_2020_main = glad_landcover_2020_main.where(glad_landcover_2020.eq(254), 200)

# #built up
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(250)).And(glad_landcover_2020.lte(253)), 250)


#### Get stable trees for 2020

In [17]:
glad_stable_tree_2020 = glad_landcover_2020_main.eq(25) #binary stable trees (TO CHECK - height definititions)

glad_stable_tree_2020 = area_stats.set_scale_property_from_image(glad_stable_tree_2020,glad_landcover_2020)

glad_stable_tree_2020_area_km2 = area_stats.binary_to_area_km2(glad_stable_tree_2020)

glad_stable_tree_2020_area_km2 = area_stats.set_scale_property_from_image(glad_stable_tree_2020_area_km2,glad_landcover_2020)

In [18]:
# ##checks
# Map = geemap.Map()
# Map.addLayer(glad_stable_tree_2020,{'min':0,'max':1,'palette':["white","green"]},"glad_stable_tree_2020")
# Map

#### Fetch: ESRI 10m Annual Land Use Land Cover (2017-2022)¶
- Link: https://gee-community-catalog.org/projects/S2TSLULC/?h=esri

- Overview: Time series of annual global maps of land use and land cover (LULC). It currently has data from 2017-2021. The maps are derived from ESA Sentinel-2 imagery at 10m resolution. Each map is a composite of LULC predictions for 9 classes throughout the year in order to generate a representative snapshot of each year. This dataset was generated by Impact Observatory, who used billions of human-labeled pixels (curated by the National Geographic Society) to train a deep learning model for land classification. This map uses an updated model from the 10-class model and combines Grass(formerly class 3) and Scrub (formerly class 6) into a single Rangeland class (class 11). The original Esri 2020 Land Cover collection uses 10 classes (Grass and Scrub separate) and an older version of the underlying deep learning model. 

- Reference:  Karra, Kontgis, et al. “Global land use/land cover with Sentinel-2 and deep learning.” IGARSS 2021-2021 IEEE International Geoscience and Remote Sensing Symposium. IEEE, 2021.
- Legend (remapped values and hex code):
- 1	Water	#1A5BAB
- 2	Trees	#358221
- 3	Flooded Vegetation	#87D19E
- 4	Crops	#FFDB5C
- 5	Built Area	#ED022A
- 6	Bare Ground	#EDE9E4
- 7	Snow/Ice	#F2FAFF
- 8	Clouds	#C8C8C8
- 9	Rangeland	#C6AD8D

In [19]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS");

esri_lulc10_2020 = esri_lulc10.filterDate('2020-01-01','2020-12-31').map(
    lambda image:
    image.remap([1,2,4,5,7,8,9,10,11],
                [1,2,3,4,5,6,7,8,9])).mosaic()


In [20]:
esri_trees_2020 = esri_lulc10_2020.eq(2) #get trees    NB check flooded veg class

esri_trees_2020_area_km2 = area_stats.binary_to_area_km2(esri_trees_2020)

esri_trees_2020_area_km2 = area_stats.set_scale_property_from_image(esri_trees_2020_area_km2,esri_lulc10.first(),0,verbose=True)


template_band_index:  0
scale (m):  10


In [21]:
#checks
# Map = geemap.Map()
# Map.addLayer(esri_trees_2020,{'min':0,'max':1,'palette':["white","green"]})
# Map

#### Fetch: Tropical Moist Forest by JRC
Link: https:#  www.science.org/doi/10.1126/sciadv.abe1603 (paper); https://forobs.jrc.ec.europa.eu/static/tmf/TMF_DataUsersGuide.pdf (dataset description)  

Overview: The transition map shows the spatial distribution of the tropical moist forest at the end of
the year 2022. It depicts the sequential dynamics of changes by providing transition stages
from the first year of the monitoring period to the end of the year 2022 (undisturbed forest,
degradation, deforestation, regrowth, conversion to plantations) and subclasses for each
transition class (period of disturbance, age of regrowth, several types of forest, several
types of degradation and deforestation, change types within the mangroves and tree
plantations).

Reference: C. Vancutsem, F. Achard, J.-F. Pekel, G. Vieilledent, S. Carboni, D. Simonetti, J. Gallego,
L.E.O.C. Aragão, R. Nasi. Long-term (1990-2019) monitoring of forest cover changes in the
humid tropics. Science Advances 2021


##### - See csv in this repo for legend and remap values: "parameters/Recoding_JRC_TMF_product.csv"





##### Reclassify JRC TMF data into 3 classes from TMF dataset: 1) undisturbed forest, 2) disturbed forest and 3) plantation

In [22]:
JRC_TMF_transitions_raw = ee.ImageCollection('projects/JRC/TMF/v1_2021/TransitionMap_Subtypes') # raw data

JRC_TMF_transitions = JRC_TMF_transitions_raw.mosaic() ### NB check why 2021?  



#remap to 3 classes relevant to EUDR 
JRC_TMF_transitions_remap = image_prep.remap_image_from_csv_cols(image=JRC_TMF_transitions,
                                                           csv_path="parameters/Recoding_JRC_TMF_product.csv",
                                                           from_col='JRC_TMF_original_class_value',
                                                           to_col='Remap_EUDR_value',
                                                           default_value=9999);
## 1) undisturbed forest, 2) disturbed forest and 3) plantation
JRC_TMF_undisturbed_2020 = JRC_TMF_transitions_remap.eq(1)

JRC_TMF_disturbed_2020 = JRC_TMF_transitions_remap.eq(2)

JRC_TMF_plantation  = JRC_TMF_transitions_remap.eq(3)

JRC_TMF_undisturbed_2020_area_km2 =area_stats.binary_to_area_km2(JRC_TMF_undisturbed_2020)
JRC_TMF_disturbed_2020_area_km2  = area_stats.binary_to_area_km2(JRC_TMF_disturbed_2020)
JRC_TMF_plantation_area_km2  = area_stats.binary_to_area_km2(JRC_TMF_plantation)

JRC_TMF_undisturbed_2020_area_km2 = area_stats.set_scale_property_from_image(JRC_TMF_undisturbed_2020_area_km2,
                                                                  JRC_TMF_transitions_raw.first())
JRC_TMF_disturbed_2020_area_km2 = area_stats.set_scale_property_from_image(JRC_TMF_disturbed_2020_area_km2,
                                                                  JRC_TMF_transitions_raw.first())
JRC_TMF_plantation_area_km2 = area_stats.set_scale_property_from_image(JRC_TMF_plantation_area_km2,
                                                                  JRC_TMF_transitions_raw.first())
                                     
print (JRC_TMF_undisturbed_2020_area_km2.get("scale").getInfo())
# JRC_TMF_transitions_main = JRC_TMF_transitions.where((JRC_TMF_transitions.gte(10)).And(JRC_TMF_transitions.lte(12)), 10) 

30.000000000000004


In [23]:
# # #checks
Map = geemap.Map()


# TransitionMap_disturbed = TransitionMap_Main.where((TransitionMap_Main.eq(20)).Or(TransitionMap_Main.eq(30)),1);
# TransitionMap_disturbed = TransitionMap_disturbed.eq(1)

# Map.addLayer(TransitionMap_disturbed,{'min':0, 'max': 1, 'palette': ["white","lightGreen"]}, "TransitionMap_disturbed",1,1);


# ### Map.addLayer(JRC_TMF_transitions_main.eq(10).randomVisualizer(),{},'TMF-simple')

# Map.addLayer(JRC_TMF_undisturbed_2020,{'min':0,'max':1,'palette':["white","green"]},'JRC_TMF_undisturbed_2020')
# Map.addLayer(JRC_TMF_transitions_remap,{'min':0,'max':3,'palette':["white","darkgreen","lightgreen","orange"]},'JRC_TMF_transitions_remap')

# Map

#### Fetch: Global Forest Change 
Link: https://www.science.org/doi/10.1126/science.1244693 (paper); https://storage.googleapis.com/earthenginepartners-hansen/GFC-2022-v1.10/download.html (user notes v1.10)

Citation: Hansen, M. C., P. V. Potapov, R. Moore, M. Hancher, S. A. Turubanova, A. Tyukavina, D. Thau, S. V. Stehman, S. J. Goetz, T. R. Loveland, A. Kommareddy, A. Egorov, L. Chini, C. O. Justice, and J. R. G. Townshend. "High-Resolution Global Maps of 21st-Century Forest Cover Change." Science 342 (15 November): 850-53. 10.1126/science.1244693.

Overview: Results from time-series analysis of Landsat images in characterizing global forest extent and change from 2000 through 2022.


N.B. Next GFC update should give tree cover in 2020. Currently showing tree cover is for 2000 with loss pixels between 2001 and 2020 removed. 


In [24]:
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

gfc_treeCover_2000 = gfc.select(['treecover2000']) #get tree cover in 2000

gfc_loss_2001_2020 = gfc.select(['lossyear']).lte(20) # get loss pixels since 2000 and up to and including 2020

gfc_treeCover_2020 = gfc_treeCover_2000.where(gfc_loss_2001_2020.eq(1),0) # remove loss from original tree cover ot approximate remaining percentage cover


##### Get treecover in 2020

In [25]:
gfc_treeCover_2020_binary=gfc_treeCover_2020.gt(10) #FAO 10% definition...

## TO DO think about gain and 2010 cover

gfc_treeCover_2020_area_km2 = area_stats.binary_to_area_km2(gfc_treeCover_2020_binary)

gfc_treeCover_2020_area_km2 = area_stats.set_scale_property_from_image(gfc_treeCover_2020_area_km2,gfc)


In [26]:
##checks
# Map = geemap.Map()
# Map.addLayer(gfc_treeCover_2020_binary,{'min':0,'max':1,'palette':["white","green"]})
# Map

#### Fetch: Primary humid forest GLAD/UMD
Link: https://developers.google.com/earth-engine/datasets/catalog/UMD_GLAD_PRIMARY_HUMID_TROPICAL_FORESTS_v1

Citation: Turubanova S., Potapov P., Tyukavina, A., and Hansen M. (2018) Ongoing primary forest loss in Brazil, Democratic Republic of the Congo, and Indonesia. Environmental Research Letters. https://doi.org/10.1088/1748-9326/aacd1c

Overview: Created by the UMD GLAD team. The primary forest extent was mapped for the year 2001 at a spatial resolution of 30 meters using globally acquired, free-of-charge, and consistently processed Landsat imagery

In [27]:
primary_HT_forests_2001_raw = ee.ImageCollection('UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1')

#get band and mosaic
primary_HT_forests_2001 = primary_HT_forests_2001_raw.select("Primary_HT_forests").mosaic().selfMask();

#remove GFC loss pixels from 2001-2020 (as previous technique with GFC, above)
primary_HT_forests_2020 = primary_HT_forests_2001.where(gfc_loss_2001_2020.eq(1),0)#.selfMask()


In [28]:
primary_HT_forests_2020_area_km2 = area_stats.binary_to_area_km2(primary_HT_forests_2020)

primary_HT_forests_2020_area_km2 = area_stats.set_scale_property_from_image(primary_HT_forests_2020_area_km2,primary_HT_forests_2001_raw.first(),0,verbose=True)


template_band_index:  0
scale (m):  27.829872698318393


In [29]:
# # checks
# Map = geemap.Map()

# # Map.addLayer(Primary_HT_forests_2001, {'min': 0,'max': 1.0,'palette':['White",'Blue']}, 'Primary HT forests: 2001');
# Map.addLayer(primary_HT_forests_2020, {'min': 0,'max': 1.0,'palette':['White','Green']}, 'Primary HT forests: 2020');

# Map

#### Fetch: JAXA Forest/non-forest maps


Links: https://www.eorc.jaxa.jp/ALOS/en/dataset/fnf_e.htm; https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF4

Citation:
Masanobu Shimada, Takuya Itoh, Takeshi Motooka, Manabu Watanabe, Shiraishi Tomohiro, Rajesh Thapa, and Richard Lucas, "New Global Forest/Non-forest Maps from ALOS PALSAR Data (2007-2010)", Remote Sensing of Environment, 155, pp. 13-31, December 2014. doi:10.1016/j.rse.2014.04.014.

Overview: The global forest/non-forest map (FNF) was produced the years 2017-2020. are generated by classifying the SAR image (backscattering coefficient) in the global 25m resolution PALSAR-2/PALSAR SAR mosaic so that strong and low backscatter pixels are assigned as "forest" and "non-forest", respectively. Here, "forest" is defined as the natural forest with the area larger than 0.5 ha and forest cover over 10%. This definition is the same as the Food and Agriculture Organization (FAO) definition. Since the radar backscatter from the forest depends on the region (climate zone), the classification of Forest/Non-Forest is conducted by using a region-dependent threshold of backscatter. The classification accuracy is checked by using in-situ photos and high-resolution optical satellite images.

Global 4-class
- 1 	 #00b200	Dense Forest
- 2 	 #83ef62	Non-dense Forest
- 3 	 #ffff99	Non-Forest
- 4 	 #0000ff	Water

In [30]:
JAXA_forestNonForest_raw = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4');

JAXA_forestNonForest_2020 =  JAXA_forestNonForest_raw.filterDate('2020-01-01', '2020-12-31').select('fnf').mosaic();

#select all trees (i.e. both dense and non-dense forest classes)
JAXA_forestNonForest_2020_binary = JAXA_forestNonForest_2020.lte(2)

In [31]:
JAXA_forestNonForest_2020_area_km2 = area_stats.binary_to_area_km2(JAXA_forestNonForest_2020_binary)

JAXA_forestNonForest_2020_area_km2 = area_stats.set_scale_property_from_image(JAXA_forestNonForest_2020_area_km2,JAXA_forestNonForest_raw.first(),0,verbose=True)

template_band_index:  0
scale (m):  24.73766462072746


In [32]:
## checks
# Map = geemap.Map()

# ## Map.addLayer(JAXA_forestNonForest_2020, {'min': 1,'max': 4,'palette': ['00b200','83ef62','ffff99','0000ff']}, 'JAXA Forest/Non-Forest 2020 - 4 classes');

# Map.addLayer(JAXA_forestNonForest_2020_binary,{'min': 0,'max': 1.0,'palette':['White','Green']}, 'JAXA Forest/Non-Forest 2020');

# Map

#### Fetch: oil palm data
Link: https://developers.google.com/earth-engine/datasets/catalog/BIOPAMA_GlobalOilPalm_v1;  https://essd.copernicus.org/articles/13/1211/2021/

Citation: Adrià, Descals, Serge, Wich, Erik, Meijaard, David, Gaveau, Stephen, Peedell, & Zoltan, Szantoi. (2021, January 27). High resolution global industrial and smallholder oil palm map for 2019 (Version v1). Zenodo. doi:10.5281/zenodo.4473715

Overview: The dataset is a 10m global industrial and smallholder oil palm map for 2019. It covers areas where oil palm plantations were detected. The classified images are the output of a convolutional neural network based on Sentinel-1 and Sentinel-2 half-year composites.

Data selection used in stats: 1 & 2

- Value	Color	Description
- 1	#ff0000	Industrial closed-canopy oil palm plantations
- 2	#ef00ff	Smallholder closed-canopy oil palm plantations
- 3	#696969	Other land covers and/or uses that are not closed-canopy oil palm. 

In [33]:
# Import the dataset; a collection of composite granules from 2019.
oil_palm_descals_raw = ee.ImageCollection('BIOPAMA/GlobalOilPalm/v1');

# Select the classification band and mosaic all of the granules into a single image.
oil_palm_descals_mosaic = oil_palm_descals_raw.select('classification').mosaic();

# Visualisation only - not needed: create a mask to add transparency to non-oil palm plantation class pixels.
mask = oil_palm_descals_mosaic.neq(3);

mask = mask.where(mask.eq(0), 0.6); #not sure about this - from online (check)

oil_palm_descals_binary = oil_palm_descals_mosaic.lte(2) #choosing to ignore



In [34]:
oil_palm_descals_binary_area_km2 = area_stats.binary_to_area_km2(oil_palm_descals_binary)

oil_palm_descals_binary_area_km2 = area_stats.set_scale_property_from_image(oil_palm_descals_binary_area_km2,oil_palm_descals_raw.first(),0,verbose=True)



template_band_index:  0
scale (m):  10


In [35]:
# # checks
# classificationVis = {#   'min': 1,#   'max': 3,   'palette': ['ff0000','ef00ff', '696969'] }; # for differen
# Map = geemap.Map()

# # Display the data on the map.
# # Map.addLayer(oil_palm_descals_mosaic.updateMask(mask),
# #              classificationVis, 'Oil palm plantation type', 1,1);

# Map.addLayer(oil_palm_descals_binary,
#              {'min':0,'max':1,'palette':["white","orange"]}, 'Oil palm plantation binary', 1,1);

# Map.setCenter(-3.0175, 5.2745,12);
# Map

#### Fetch FDaP Palm probability map (Forest Data Partnership unpublished product): Indonesia and Malaysia
- model produced as part of the FDaP project. 
- NB preliminary output, recieved from Nick clinton (Google) on 16/11/2023

In [36]:
FDaP_palm_2020_model_raw = ee.ImageCollection("projects/forestdatapartnership/assets/palm/palm_2020_model_20231026");
FDaP_palm_2020_model = FDaP_palm_2020_model_raw.mosaic().gt(0.9).selfMask()


In [37]:
FDaP_palm_2020_model_area_km2 = area_stats.binary_to_area_km2(FDaP_palm_2020_model)
FDaP_palm_2020_model_area_km2 = area_stats.set_scale_property_from_image(FDaP_palm_2020_model_area_km2,FDaP_palm_2020_model_raw.first(),0,verbose=True)



template_band_index:  0
scale (m):  10


In [38]:
## Checks
# Map = geemap.Map()
# Map.addLayer(palm_2020_model, {'min':0,'max':1,'palette': ['white','orange']}, 'palm_2020_model');
# Map

#### Cocoa maps: Côte d'Ivoire and Ghana
Link(s): https://nk.users.earthengine.app/view/cocoa-map; https://www.nature.com/articles/s43016-023-00751-8

Citation: Kalischek, N., Lang, N., Renier, C. et al. Cocoa plantations are associated with deforestation in Côte d’Ivoire and Ghana. Nat Food 4, 384–393 (2023). https://doi.org/10.1038/s43016-023-00751-8

Overview: Cocoa farming maps for Côte d'Ivoire and Ghana at a resolution of 10m. The maps are based on Sentinel-2 images taken between January 2019 and December 2021, which are interpreted with an ensemble of convolutional neural networks (CNN), each trained on a large corpus of geo-referenced ground truth cocoa farms and non-cocoa sites, including more than 100,000 farms. Each model predicts a binary cocoa map and the final map is computed by taking the average, indicating the degree of agreement among the models. Additionally, we compute the best threshold at 0.65 according to a held-out validation set, i.e. all values above 65 % in the probability map are classified as cocoa and all others as non-cocoa areas.

- NB imagery years usd for model include 2021 ( the followign year) so some LU conversion may have occurred (in either direction) since 2020 (a key year of interest for EUDR) 
- Using suggested threshold dataset of >0.65 from GEE script
- Chose to not include al;ternative cocoa map for this area by Abu et al. (2021), map as independent validation suggested their model performed poorly (pers. comm. M. Sassen, Nov 2023)

In [39]:
# cocoa_map_kalischek = ee.ImageCollection('projects/ee-nk-cocoa/assets/cocoa_map');

cocoa_map_kalischek_threshold = ee.Image('projects/ee-nk-cocoa/assets/cocoa_map_threshold_065');

In [40]:
cocoa_map_kalischek_threshold_area_km2 = area_stats.binary_to_area_km2(cocoa_map_kalischek_threshold)
cocoa_map_kalischek_threshold_area_km2 = area_stats.set_scale_property_from_image(cocoa_map_kalischek_threshold_area_km2,cocoa_map_kalischek_threshold,0,verbose=True)


template_band_index:  0
scale (m):  10.080003101582253


In [41]:
## Checks
# Map = geemap.Map()
# Map.addLayer(cocoa_map_kalischek_threshold, {'min':0,'max':1,'palette': ['white','brown']}, 'palm_2020_model');
# Map

#### Fetch: World Database on Protected Areas (WDPA)
Link: www.protectedplanet.net

Citation: UNEP-WCMC and IUCN (year), Protected Planet: The World Database on Protected Areas (WDPA) [On-line], [insert month/year of the version used], Cambridge, UK: UNEP-WCMC and IUCN Available at: www.protectedplanet.net.

Overview: The World Database on Protected Areas (WDPA) is the most up-to-date and complete source of information on protected areas, updated monthly with submissions from governments, non-governmental organizations, landowners, and communities. It is managed by the United Nations Environment Programme's World Conservation Monitoring Centre (UNEP-WCMC) with support from IUCN and its World Commission on Protected Areas (WCPA).

WDPA User Manual. For details including methodologies,standards, data providers, metadata field definitions and descriptions, refer to the WDPA User Manual.
 

Processing info:

Protected areas converted to raster (i.e. an image) at high resolution (approx 30m). 

Rational: avoids use of hefty polygon intersections just to show presence in ROI and fits in current raster based framework.

Filtering and preparation. Currently based on use for protected area statistics.
- date: this is using "current" data (it's updated monthly) - assuming if plot is in a protected area they need to know even if after 2020.
- point data: currently buffering by reported area where this exists.
- removed sites: removed all but designated sites; removed Man and Biosphere reserves (for detail see: WDPA Manual)

- TO DO: add OECMs (Other Effective area-based Conservation Mechanisms). Rationale: these are now being recognised for Target 3 of the Global Biodiversity Framework 
- TO DO: check filtering steps - especially how to handle buffered points - seperate column as estimated boundary only?


In [42]:
wdpa_pnt = ee.FeatureCollection("WCMC/WDPA/current/points");

wdpa_poly = ee.FeatureCollection("WCMC/WDPA/current/polygons");

#apply filters and merge polygon with buffered points  
wdpa_filt = WDPA_prep.filterWDPA(wdpa_poly).merge(WDPA_prep.filterWDPA(wdpa_pnt).filter(ee.Filter.gt('REP_AREA', 0)).map(WDPA_prep.bufferByArea));
#turn into image (no crs etc set currently)
wdpa_overlap = wdpa_filt.reduceToImage(['STATUS_YR'],'min');  #make into raster - remove mask if want 0s

#make binary
wdpa_binary = wdpa_overlap.lt(2070).unmask()


##### Setting projection and resolution etc (based on GFC scale as template)

In [43]:
#reproject based on gfc data (approx 30m res which should be easily)
crs_template = gfc.select(0).projection().crs().getInfo()

wdpa_binary_reproj = wdpa_binary.reproject(
  crs= crs_template,
  scale= area_stats.get_scale_from_image(gfc),
).int8()

protected_areas_WDPA_area_km2 = area_stats.binary_to_area_km2(wdpa_binary_reproj)

protected_areas_WDPA_area_km2 = area_stats.set_scale_property_from_image(protected_areas_WDPA_area_km2,gfc,0,verbose=True)


template_band_index:  0
scale (m):  27.829872698318393


In [44]:
# ##checks

# Map = geemap.Map()

# Map.addLayer(wdpa_filt,"","filtered wdpa_poly buff pnt",1,1)

# # image version takes a while to show
# Map.addLayer(wdpa_binary_reproj, {min:0, max:1, 'palette':['white','purple']},'wdpa_binary_reproj',1,1); #visualise raster - with count of overlaps

# # Map.set
# Map

#### Fetch: Key Biodiversity Areas (KBAs)
Link: https://www.keybiodiversityareas.org/kba-data/request; 
https://www.keybiodiversityareas.org/termsofservice

Citation: BirdLife International ([year e.g. 2023]). The World Database of Key Biodiversity Areas. Developed by the KBA Partnership: BirdLife International, International Union for the Conservation of Nature, Amphibian Survival Alliance, Conservation International, Critical Ecosystem Partnership Fund, Global Environment Facility, Re:wild, NatureServe, Rainforest Trust, Royal Society for the Protection of Birds, Wildlife Conservation Society and World Wildlife Fund. Available at www.keybiodiversityareas.org. [Accessed (please insert date of download dd/mm/yyyy)].

Overview: Key Biodiversity Areas, which are among the most incredible and diverse places on Earth for nature, from deserts to the middle of the ocean, are sites of global importance to the planet’s overall health and the persistence of biodiversity. The Key Biodiversity Area Partnership - an ambitious partnership of 13 global conservation organizations - is helping prevent the rapid loss of biodiversity by supporting nationally led efforts to identify these places on the planet that are critical for the survival of unique plants and animals, and the ecological communities they comprise.

By mapping these most important sites on Earth, and providing information about the wildlife living there, private industry, governments and other stakeholders can make the best decisions about how to manage that land (or waters), where to avoid development, and how best to conserve and protect the animals and plants for which the sites are so important.

NB
- Useful flag as these areas typically become protected (large overlap with existing protected areas whcih take a w hwile to be designated) 
- The last populations of species are found in a subset of KBAs (i.e., Alliance for Zero Extinction (AZE) sites)
- KBAs have rigourous inclusion criteria and typically have active management for protecting species they contain

In [45]:
kbas_2023_poly = ee.FeatureCollection("projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/raw/KBAsGlobal_2023_March_01_POL");##uploaded - may need rights

kba_2023_overlap = kbas_2023_poly.reduceToImage(['SitRecID'],'count').selfMask()  #make into raster - remove mask if want 0s

kba_2023_binary = kba_2023_overlap.gte(0)

##### Setting projection and resolution etc (based on GFC scale as template)

In [46]:
#reproject based on gfc data
crs_template = gfc.select(0).projection().crs().getInfo()

kba_2023_binary_reproj = kba_2023_binary.reproject(
  crs= crs_template,
  scale= area_stats.get_scale_from_image(gfc),
).int8()

kba_2023_area_km2 = area_stats.binary_to_area_km2(kba_2023_binary_reproj)

kba_2023_area_km2 = area_stats.set_scale_property_from_image(kba_2023_area_km2,gfc,0,verbose=True)



template_band_index:  0
scale (m):  27.829872698318393


In [47]:
# # ##checks
# Map = geemap.Map()
# # Map.addLayer(kbas_2023_poly,"",'kbas_2023_poly',0,1); 
# # Map.addLayer(kba_2023_overlap, {'min':0, 'max':5, 'palette':['blue','red']},'kba_overlap',0,1); #visualise raster - with count of overlaps
# Map.addLayer(kba_2023_binary, {'min':0, 'max':1, 'palette':['white','black']},'kba_binary',1,1); #binary raster

# Map

#### Fetch: RADD forest disturbance alert

Link: http://radd-alert.wur.nl; https://gee-community-catalog.org/projects/radd/

Citation: Reiche J, Mullissa A, Slagter B, Gou Y, Tsendbazar N-E, Odongo-Braun C, Vollrath A, Weisse MJ, Stolle F, Pickens A, Donchyts G, Clinton N, Gorelick N, and Herold M (2021). https://doi.org/10.1088/1748-9326/abd0a8

Overview:
RADD - RAdar for Detecting Deforestation - Near real-time disturbances in humid tropical forest based on Sentinel-1 at 10m spatial scale. 
NB only primary humid tropical forest of South America (13 countries), Central America (6 countries), Africa (25 countries), insular Southeast Asia (5 countries) and Pacific (1 country).
A new forest disturbance alert is triggered based on a single observation from the latest Sentinel-1 C-band radar image. Subsequent observations are used to iteratively update the forest disturbance probability, increase confidence and confirm or reject the alert. Alerts are confirmed within a maximum 90-day period if the forest disturbance probability is above 97.5% (high confidence alerts). Unconfirmed alerts (low confidence alerts) are provided for forest disturbance probabilities above 85%. The date of the alert is set to the date of the image that first triggered the alert.


Approach for deforestations risk proxy: run stats for areas around each input feature and 5km buffer. 
Select only recent forest loss (i.e. in the last 2 years) based on todays date.

Possible data alternatives: use GFC loss (not as recent); use combination of loss and alerts; use off-shelf deforestation risk product (no buffer needed for this).

Instead of buffer and recency, use actual rates such as here https://code.earthengine.google.com/ab1e640b81a107b796718f285a56422a


In [48]:
# Getting today's date
ee_now =ee.Date(datetime.now())#.format()
# print(ee_now.getInfo())

# Calculate the start date
start_date = ee_now.advance(how_many_days_back, "day")

# Needs to be in yyDDD format and needs to be a number, so need to parse too
start_date_yyDDD = ee.Number.parse(start_date.format('yyDDD'))

print("start_date_yyDDD", start_date_yyDDD.getInfo())


start_date_yyDDD 21326


In [49]:
# Define the Image Collection
radd = ee.ImageCollection('projects/radar-wur/raddalert/v1')

# Forest baseline (from Primart HT forests data)
forest_baseline = ee.Image(radd.filterMetadata('layer', 'contains', 'forest_baseline')
    .mosaic())
 
# Latest RADD alert
latest_radd_alert = ee.Image(radd.filterMetadata('layer', 'contains', 'alert')
    .sort('system:time_end', False)
    .mosaic())

# Update mask for RADD alert to be within primary forest (TO CHECK maybe unnecessry step)
latest_radd_alert_masked = latest_radd_alert.select('Alert')#.updateMask(forest_baseline)

# Mask confirmed alerts #TO CHECK do we want to be more conservative?
latest_radd_alert_masked_confirmed = latest_radd_alert_masked.unmask().eq(3)

# Update mask for confirmed alerts by date
latest_radd_alert_confirmed_recent = latest_radd_alert.select('Date').gte(start_date_yyDDD).selfMask()


In [50]:
latest_radd_alert_confirmed_recent_area_km2 = area_stats.binary_to_area_km2(latest_radd_alert_confirmed_recent)

latest_radd_alert_confirmed_recent_area_km2 = area_stats.set_scale_property_from_image(latest_radd_alert_confirmed_recent_area_km2,radd.first(),0,verbose=True)


template_band_index:  0
scale (m):  10


In [51]:
# # #checks
# Map = geemap.Map()

# # # # Add Forest baseline layer to the map
# Map.addLayer(forest_baseline, {'palette': ['darkgreen']}, 'Forest baseline', 0, 1)

# Map.addLayer(latest_radd_alert_confirmed_recent,
#     {'min': 0, 'max': 1, 'palette': ['white', 'red']},
#     'RADD alert masked confirmed - recent (i.e., filtered by date)', 1, 1)


# Map

#### Set properties for which images need special treatment

In [52]:
#deforestation alerts
# set property so run stats for a buffer around site; 
# and show presence only as output 
latest_radd_alert_confirmed_recent_area_km2 = latest_radd_alert_confirmed_recent_area_km2.setMulti(
    {"alerts_buffer":1,"presence_only_flag":1})

#important sites: 1) protected areas and 2) KBAs (likely future protectred areas) 
# show presence only as output 
protected_areas_WDPA_area_km2 = protected_areas_WDPA_area_km2.set("presence_only_flag",1)

kba_2023_area_km2 = kba_2023_area_km2.set("presence_only_flag",1)



#### Create dictionary of images and image names
- prep for reduceRegions statistics so name of datasets/image is added to area stats
- sets "system:index" property of each image
- result is an image collection with 

In [53]:
image_names_dict0 = {"GFC_Treecover_2020":gfc_treeCover_2020_area_km2,
               "TMF_undisturbed_forest_2020":JRC_TMF_undisturbed_2020_area_km2,
                     "TMF_disturbed_forest_2020": JRC_TMF_disturbed_2020_area_km2,
                     "TMF_plantation":JRC_TMF_plantation_area_km2,
                "ESRI_Trees_2020":esri_trees_2020_area_km2,
                "GLAD_Stable_Tree 2020":glad_stable_tree_2020_area_km2,
                "Oil_Palm_Descals": oil_palm_descals_binary_area_km2,
                     "FDaP_palm_2020": FDaP_palm_2020_model_area_km2,
                 "Cocoa_plantations_Kalischek": cocoa_map_kalischek_threshold_area_km2,
                "Local_RADD_alerts":latest_radd_alert_confirmed_recent_area_km2,
                "Protected_area":protected_areas_WDPA_area_km2,
                "Key_Biodiversity_Area": kba_2023_area_km2,
                  "Primary_HT_forests_2020": primary_HT_forests_2020_area_km2,
                   "JAXA_Forest_non_forest_2020":JAXA_forestNonForest_2020_area_km2
              }

## can be cleaned up...

#create empty dictionary to be populated
image_names_dict={} 
    
#set image names ("system:index") from keys in dictionary, and store as new one
for i in range(len(image_names_dict0)):
    dataset_name = (list(image_names_dict0.keys())[i]) #get dataset name
    image = (list(image_names_dict0.values())[i]) #get image
    updated_image=image.set("system:index",dataset_name) #set dataset name as image name i.e., "system:index"
    instance={dataset_name:updated_image} #combine
    image_names_dict.update(instance) #update into new dictionary

del image_names_dict0 # remove old dictionary

#make into a new image collection
images_IC = ee.ImageCollection(list(image_names_dict.values()))
                      
##checks
print ("number of images: ",len(image_names_dict))

images_IC

number of images:  14


#### Compute statistics

Calculating zonal statistics for continuous data (e.g tree cover) within polygon(s)

step 1: save as combined CSV in long format NB needs speeding up e.g., reduceRegions and dataframes

##### Mapping over image collection with reduce regions

In [54]:
# get the start time
st = time.time()
print ("processing stats...")


#get stats for roi (except alerts)
fc_stats_combined = area_stats.reduceStatsIC(roi,
                                  images_IC.filter(ee.Filter.neq("alerts_buffer",1)),
                                  reducer_choice)# all but alerts
#get stat for buffer (alerts only)
fc_stats_combined_buffer = area_stats.reduceStatsIC(roi_alerts_buffer,
                                         images_IC.filter(ee.Filter.eq("alerts_buffer",1)),
                                         reducer_choice) #alerts only

#combine stats from roi and buffer
fc_stats_combined_all = fc_stats_combined.merge(fc_stats_combined_buffer) # combining alerts with others into one feature collection

# convert to Pandas Dataframe
df_combined = geemap.ee_to_pandas(fc_stats_combined_all) # limit of 5000 (unlikely to need more but i have code for it if needed)

# export dataframe to csv
df_combined.to_csv(path_or_buf=out_file_long,header=True,index=False)

# get the execution time
elapsed_time = time.time() - st

print('Total execution time:', elapsed_time, 'seconds')

#checks
df_combined

processing stats...
Total execution time: 8.811205387115479 seconds


,dataset_name,sum,count,Geo_id,Geometry_area_km2
0,GFC_Treecover_2020,0.051586,137,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,0.083162
1,GFC_Treecover_2020,0.015586,41,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...,0.019902
2,GFC_Treecover_2020,0.026586,68,b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0...,0.038155
3,GFC_Treecover_2020,0.016037,65,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,0.036282
4,GFC_Treecover_2020,0.003966,40,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,0.019471
...,...,...,...,...,...
149,Local_RADD_alerts,0.162493,1652,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,0.166761
150,Local_RADD_alerts,0.000000,0,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,0.313534
151,Local_RADD_alerts,0.000000,0,1a41a309ae2387f36a604c9a6c81887e64357a7f61d228...,0.019736
152,Local_RADD_alerts,0.000000,0,1a4472dc40700ef33f931863f58d444f243d6441861667...,0.127822


##### Reformat
- long to wide
- convert to proportions 
- set presence only flags

In [65]:
#add proprtion column
df_combined["proportion"] = (df_combined["sum"]/df_combined[geometry_area_column])

#convert to wide format (one row per geo_id)
df_wide_format = df_combined.pivot_table(index=[geo_id_column],columns=['dataset_name'],values=['proportion'])

#tidy unwanted headers etc
tidy_tables.tidy_dataframe_after_pivot(df_wide_format) #runs in place so no need to assign

#list images with with presence_only_flag property 
flag_list = images_IC.filter(ee.Filter.eq("presence_only_flag",1)).aggregate_array("system:index").getInfo()

# convert pos results values to "True" for specific columns
for column in flag_list: 
    df_wide_format[column]=np.where(df_wide_format[column]>0,"True","-")

# tidy output - decimal places
df_wide_format = df_wide_format.round(decimals=3, out=None)

#tidy (i.e. remove underscores)
df_wide_format.columns = df_wide_format.columns.str.replace('_', ' ')

#export wide format csv
df_wide_format.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)

#checks
flag_list
df_wide_format


output csv:  /home/sepal-user/fdap/stats_wide_format_continuous_data.csv


,Cocoa plantations Kalischek,ESRI Trees 2020,FDaP palm 2020,GFC Treecover 2020,GLAD Stable Tree 2020,JAXA Forest non forest 2020,Key Biodiversity Area,Local RADD alerts,Oil Palm Descals,Primary HT forests 2020,Protected area,TMF disturbed forest 2020,TMF plantation,TMF undisturbed forest 2020
Geo_id,,,,,,,,,,,,,,
0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918,0.000,0.996,0.000,0.620,0.638,0.952,True,True,0.000,0.000,True,0.102,0.000,0.003
1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7,0.000,0.995,0.039,0.002,0.994,0.849,-,-,0.000,0.000,-,0.133,0.000,0.000
1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45,0.000,0.995,0.684,0.828,0.988,0.996,-,-,0.000,0.000,-,0.394,0.000,0.600
88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907,0.000,0.995,0.000,0.204,0.877,0.995,-,-,0.000,0.000,-,0.341,0.000,0.000
8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a,0.000,0.995,0.509,0.535,0.981,0.959,-,-,0.000,0.000,-,0.668,0.000,0.308
97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb,0.000,0.996,0.000,0.995,0.890,0.995,-,True,0.000,0.868,True,0.049,0.000,0.947
b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3,0.000,0.995,0.000,0.697,0.900,0.995,-,-,0.000,0.000,-,0.741,0.000,0.000
b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f,0.733,0.995,0.000,0.783,0.554,0.726,-,True,0.000,0.000,True,0.235,0.000,0.000
c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d,0.000,0.056,0.820,0.004,0.831,0.051,-,-,0.984,0.000,-,0.000,0.995,0.000


#### Display layers

NB add legends


In [66]:
Map = geemap.Map()
visParams =  {'min': 0,'max': 1,'palette':['White','Green']}
for i in range(images_IC.size().getInfo()):
    
    imageNew = ee.Image(images_IC.toList(100,0).get(i))
    
    dataset_name = (list(image_names_dict.keys())[i])
    
    print ("adding image",i,"-",dataset_name)
    Map.addLayer(imageNew.gt(0).unmask(),visParams,dataset_name,0,1)
    
    
Map.addLayer(roi,{},'roi ',1,1)
# Map.addLayer(roi_alerts_buffer,{},'roi buffer zone')

print ("All layers added")    

adding image 0 - GFC_Treecover_2020
adding image 1 - TMF_undisturbed_forest_2020
adding image 2 - TMF_disturbed_forest_2020
adding image 3 - TMF_plantation
adding image 4 - ESRI_Trees_2020
adding image 5 - GLAD_Stable_Tree 2020
adding image 6 - Oil_Palm_Descals
adding image 7 - FDaP_palm_2020
adding image 8 - Cocoa_plantations_Kalischek
adding image 9 - Local_RADD_alerts
adding image 10 - Protected_area
adding image 11 - Key_Biodiversity_Area
adding image 12 - Primary_HT_forests_2020
adding image 13 - JAXA_Forest_non_forest_2020
All layers added


In [82]:
# roi

single_feature_id = roi.aggregate_array(geo_id_column).get(1).getInfo()
print(geo_id_column,single_feature_id)
single_feature = ee.Feature(roi.filter(ee.Filter.eq(geo_id_column,single_feature_id)).first())

Map.centerObject(single_feature,12)
    
Map
# # single_feature


Geo_id b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f


Map(bottom=508035.0, center=[5.673810322182612, -4.086848133220072], controls=(WidgetControl(options=['positio…

...testing area - recoding per geo-id

In [ ]:

# def reduceStatsIC (featureCollection,imageCollection,reducer_choice):
#     """Calculating summary statistics for each image in collection, within each feature in a collection""" 
  
#     def reduceStats (image):
#         scale=image.get("scale")
#         fc = ee.FeatureCollection(image.reduceRegions(collection=featureCollection,
#                                                       reducer=reducer_choice,
#                                                       scale=scale))
#         fc = fc.map(lambda feature: feature.set("dataset_name",image.get("system:index")))
#         return fc
    
#     fc_out = imageCollection.map(reduceStats).flatten()
    
#     return fc_out

# function to:
# Input one geo_id
# Create one feature
# Get statistics
# for one feature
# and one image
# Repeat for each image in collection
# Add image name as property
# Repeat for all features using parallel processing code
# Spits out feature collections client side
# Compile into a csv

# Question:
#     - should we map over image collection to make an fc, before or after statistics are run in parallel?



#### Alternative stats


In [ ]:
# # Define the class names
# class_names = ['TMF_undis', 'TMF_degrad', 'TMF_regrow', 'Defor_plantation', 'Defor_water', 'Defor_land','Defor_active','water','other_land']

# # Create a function to count pixels for each class
# def count_pixels(image, roi):
#     counts = image.reduceRegion(
#         reducer=ee.Reducer.frequencyHistogram(),
#         geometry=roi,
#         scale=90,
#         maxPixels=1e9
#     )
    
#     # Convert pixel counts to a feature with properties
#     feature = ee.Feature(None, counts)
    
#     return feature

# # Apply the count_pixels function to the classified image
# pixel_counts_feature = count_pixels(tmf, roi)

# # Create a feature collection with a single feature
# fc_pix = ee.FeatureCollection([pixel_counts_feature])


# fc_pix_df = geemap.ee_to_pandas(fc_pix)##needs more work
# fc_pix_df


### Logout (protected)

In [ ]:
# res = session.get(asset_registry_base + "/logout")
# print(res.json())
# res = session.get(user_registry_base + "/logout", cookies=session.cookies)
# session.headers.clear()

### Checking if Logged out correctly

In [ ]:
# # Confirming the logout from Asset Registry by requesting a Protected route
# req_body = {
#     "latitude": 31.47704430446457,
#     "longitude": 74.37510786779589
# }
# res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
# print(res.json())

### Get all Domains (public)

In [ ]:
# # Fetching all the domains from the User Registry
# res = session.get(asset_registry_base + "/domains")
# print(res.json())